In [ ]:
!pip install numpy
from google.colab import drive
import os
drive.mount('/content/gdrive/')
%cd gdrive/MyDrive/IA 2023

Mounted at /content/gdrive/
/content/gdrive/MyDrive/IA 2023


In [ ]:
import numpy as np

In [ ]:
filename = "/content/gdrive/MyDrive/IA 2023/VDA/samplesVDA1.txt"
x_vda = []
with open(filename) as f:
  for lines in f:
    x_vda.append(int(lines))

  x_vda = np.array(x_vda)

f.close()

print(x_vda)
print(len(x_vda))

[421 424 431 440 455 474 495 518 542 565 587 608 627 645 659 669 675 679
 678 674 669 662 652 644 635 624 615 606 597 589 582 575 570 566 562 560
 559 558 560 560 563 565 568 571 572 573 573 572 569 567 564 562 560 558
 558 557 557 557 558 559 559 559 560 561 562 563 562 562 562 561 560 559
 558 556 557 555 554 553 552 551 548 546 544 541 537 534 531 528 524 521
 518 515 512 509 507 505 502 500 498 495 494 493 492 491 492 491 492 493
 493 495 495 496 496 496 494 493 492 491 489 488 486 485 483 480 478 477
 475 472 472 470 468 466 464 463 461 460 459 458 457 458 458 457 457 457
 456 457 456 456 455 455 454]
151
